# Automated Sensing of Red vs. White Wine
_... with the cheapest sensor possible_

#### Data Science 420 BB
#### University of Washington Professional & Continuing Education

#### Homework 1: Low-cost Sensor
#### Leo Salemann, 4/11/18

# Read the Data, Have a Look
## Initial Load

In [7]:
file <- "./RedWhiteWine.csv"
all_data <- read.csv(file, header=TRUE, sep=",", stringsAsFactors = FALSE)

# If the target column, say column class, is numerical, you need to convert it to factors in R for classification problem
all_data$Class <- factor(data$Class)
head(all_data)


fixed.acidity,volatile.acidity,citric.acid,residual.sugar,chlorides,free.sulfur.dioxide,total.sulfur.dioxide,density,pH,sulphates,alcohol,quality,Class
7.4,0.70,0.00,1.9,0.076,11,34,0.9978,3.51,0.56,9.4,5,1
7.8,0.88,0.00,2.6,0.098,25,67,0.9968,3.20,0.68,9.8,5,1
7.8,0.76,0.04,2.3,0.092,15,54,0.9970,3.26,0.65,9.8,5,1
11.2,0.28,0.56,1.9,0.075,17,60,0.9980,3.16,0.58,9.8,6,1
7.4,0.70,0.00,1.9,0.076,11,34,0.9978,3.51,0.56,9.4,5,1
7.4,0.66,0.00,1.8,0.075,13,40,0.9978,3.51,0.56,9.4,5,1


## Reduce to the minimally-required columns

In [8]:
library("dplyr")

data = select(all_data, fixed.acidity, volatile.acidity, citric.acid, residual.sugar, chlorides, density, Class)
head(data)
dim(data)
summary(data)

fixed.acidity,volatile.acidity,citric.acid,residual.sugar,chlorides,density,Class
7.4,0.70,0.00,1.9,0.076,0.9978,1
7.8,0.88,0.00,2.6,0.098,0.9968,1
7.8,0.76,0.04,2.3,0.092,0.9970,1
11.2,0.28,0.56,1.9,0.075,0.9980,1
7.4,0.70,0.00,1.9,0.076,0.9978,1
7.4,0.66,0.00,1.8,0.075,0.9978,1


[1] 6497    7

 fixed.acidity    volatile.acidity  citric.acid     residual.sugar  
 Min.   : 3.800   Min.   :0.0800   Min.   :0.0000   Min.   : 0.600  
 1st Qu.: 6.400   1st Qu.:0.2300   1st Qu.:0.2500   1st Qu.: 1.800  
 Median : 7.000   Median :0.2900   Median :0.3100   Median : 3.000  
 Mean   : 7.215   Mean   :0.3397   Mean   :0.3186   Mean   : 5.443  
 3rd Qu.: 7.700   3rd Qu.:0.4000   3rd Qu.:0.3900   3rd Qu.: 8.100  
 Max.   :15.900   Max.   :1.5800   Max.   :1.6600   Max.   :65.800  
   chlorides          density       Class   
 Min.   :0.00900   Min.   :0.9871   0:4898  
 1st Qu.:0.03800   1st Qu.:0.9923   1:1599  
 Median :0.04700   Median :0.9949           
 Mean   :0.05603   Mean   :0.9947           
 3rd Qu.:0.06500   3rd Qu.:0.9970           
 Max.   :0.61100   Max.   :1.0390           

## Redefine Class 1 as Red and 0 as white [TODO]

# Make a function that quickly tests accuracy

In [ ]:
install.packages("naivebayes")
library(naivebayes)
install.packages("pROC")
library(pROC)

In [46]:
test_nbc_accuracy <- function(df) {
    # split into test and training sets
    nrows <- nrow(df)
    smp_size <- floor(0.75 * nrows)

    set.seed(8720885)
    train_ind <- sample(seq_len(nrows), size = smp_size)
    
    train <- df[train_ind, ]
    test <- df[-train_ind, ]
    
    # create the Naive Bayes Classifer (nbc)
    nbc <- naive_bayes(Class ~ ., data = train)
    
    # Make a prediction & test accuracy
    test_pred <- predict(nbc, test,type = "prob")
    
    ### junky debug stuff
    print(head (test_pred))
    ### junky debug stuff
    accuracy <- sum(test[,"Class"]==test_pred)/length(test_pred)
    print(paste("Accuracy=", round(accuracy*100,2), "%", sep=""))

    print(paste("AUC=", auc(test_pred, df$Class)))

}

In [47]:
test_nbc_accuracy(data)

                0         1
[1,] 6.104496e-05 0.9999390
[2,] 1.136686e-03 0.9988633
[3,] 8.027409e-04 0.9991973
[4,] 8.319823e-01 0.1680177
[5,] 4.743836e-04 0.9995256
[6,] 2.234235e-37 1.0000000
[1] "Accuracy=0.43%"


Warning message in roc.default(response, predictor, auc = TRUE, ...):
“'response' has more than two levels. Consider setting 'levels' explicitly or using 'multiclass.roc' instead”

ERROR: Error in roc.default(response, predictor, auc = TRUE, ...): Predictor must be numeric or ordered.


In [22]:
# Try Dropping chlorides, since it's the opposite of residual sugar

In [23]:
test_data = select(data, -chlorides)
head(test_data)

fixed.acidity,volatile.acidity,citric.acid,residual.sugar,density,Class
7.4,0.70,0.00,1.9,0.9978,1
7.8,0.88,0.00,2.6,0.9968,1
7.8,0.76,0.04,2.3,0.9970,1
11.2,0.28,0.56,1.9,0.9980,1
7.4,0.70,0.00,1.9,0.9978,1
7.4,0.66,0.00,1.8,0.9978,1


In [24]:
test_nbc_accuracy(test_data)

[1] "Accuracy=0.12%"


In [25]:
# Try Dropping Volatile acidity

In [26]:
test_data = select(test_data, -volatile.acidity)
head(test_data)

fixed.acidity,citric.acid,residual.sugar,density,Class
7.4,0.00,1.9,0.9978,1
7.8,0.00,2.6,0.9968,1
7.8,0.04,2.3,0.9970,1
11.2,0.56,1.9,0.9980,1
7.4,0.00,1.9,0.9978,1
7.4,0.00,1.8,0.9978,1


In [32]:
test_nbc_accuracy(test_data)
test_nbc_accuracy(data)
test_nbc_accuracy(all_data)
test_nbc_accuracy(select(all_data, -quality))

[1] "Accuracy=0.06%"
[1] "Accuracy=0.43%"
[1] "Accuracy=0.62%"
[1] "Accuracy=0.62%"


In [34]:


auc(test_data$, test_data$Class)

Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)


# ====================< JUNK >=====================

# Create a Naive Bayes Model, Try a Prediction

In [ ]:
nrows <- nrow(data)
smp_size <- floor(0.75 * nrows)

## set the seed to make your partition reproductible
set.seed(8720885)
train_ind <- sample(seq_len(nrows), size = smp_size)

In [ ]:
train <- data[train_ind, ]
test <- data[-train_ind, ]

In [ ]:
options(repr.plot.width=4, repr.plot.height=4)
plot(nbc)

In [ ]:
install.packages("naivebayes")
library(naivebayes)
nbc <- naive_bayes(Class ~ ., data = train)
plot(nbc)

In [ ]:
# Predict the test data
test_pred <- predict(nbc, test,type = "prob")

accuracy <- sum(test[,"Class"]==test_pred)/length(test_pred)
print(paste("Accuracy=", round(accuracy*100,2), "%", sep=""))

## Pull out a handy plotting function from last quarter

In [ ]:
require(ggplot2)
plot.density = function(df, col, xlabel, title) {
    ggplot(df, aes(col)) + 
          geom_bar() + 
          xlab(xlabel) + 
          ggtitle(title) + 
          theme(plot.title = element_text(hjust = 0.5, size=20, face="bold"),
               axis.title.x = element_text(size=15))
}

In [ ]:
options(repr.plot.width=12, repr.plot.height=4)
plot.density(data, data$Class, "1 = Red?, 2 = White?", "Wine Color")

## Try some Fancy Matrix Plotting

In [ ]:
require(GGally)

plot.matrix = function(df, cols){
    options(repr.plot.width=8, repr.plot.height=8)
    ggpairs(data=df,
            columns=cols,
            upper = list(continuous = "density"),
            lower = list(continuous = "density"),
            cardinality_threshold = 130)
}

In [ ]:
plot.matrix(data, c(1:6,7))

In [ ]:
prediction = predict(nbc, test, type= "prob")
head(prediction)